In [1]:
import numpy as np
import pandas as pd
import sklearn as skl
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime
import random

In [112]:
with open('C:/Users/Robin_Koester/Desktop/GANs/Energy_Data/DayAhead_hour_20152017.csv', newline = '') as csvfile:
    datareader = csv.reader(csvfile, delimiter = ';')
    
    data_array = np.array(list(datareader))    
    
price_array = np.array([np.array(data_array[i]) for i in range(1,len(data_array))])

prices = np.array(list(map(lambda it: float(price_array[it,1]) , range(1,len(price_array))) ))

date_array = np.array(list(map(lambda it: datetime.strptime(price_array[it,0],'%d.%m.%Y %H:%M') , range(1,len(price_array))) ))

day_of_week_array = np.array(list(map(lambda date: datetime.weekday(date), date_array)))

hour_array = np.array(list(map(lambda date: datetime.time(date).hour, date_array)))

In [194]:
tf.reset_default_graph()

# hyperparameters 

mb_size = 1000                    # size of mini batches, can be randomly sampled
time_predict_dim = 24
time_conditional_dim = 24       # the whole snippet of the time series should comprise two day
z_dim = 0                      # number of random inputs
h_dim = 128                     # number of neurons in the hidden layer: ohne hidden layer in the each network
lr = 1e-3                       # learning rate
d_steps = 3
no_iterations = 50000                # number of steps for the training algorithm
reg_scale = 1e-2

gen_input_dim = z_dim + time_predict_dim + 2           # conditional inputs and z_dim random inputs  (input of the generator net)
discr_input_dim = time_predict_dim + time_conditional_dim + 2

# definition of functions

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)   # return the respective initialized weights


def log(x):
    return tf.log(x + 1e-8)


def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def next_batch(size):
    ind_vec = list(range(len(prices)-X_dim))
    ind_vec_shuffled = random.sample(ind_vec,len(ind_vec))

    mb_ind = ind_vec_shuffled[0:(size)]
    
    mb_prices_day1 = np.array(list(map(lambda mb_first_ind: prices[mb_first_ind:(mb_first_ind+(X_dim-24))], mb_ind)))   
    mb_prices_day2 = np.array(list(map(lambda mb_first_ind: prices[(mb_first_ind+(X_dim-24)):(mb_first_ind+X_dim)], mb_ind)))
    
    weekday = np.array(list(map(lambda mb_first_ind: day_of_week_array[mb_first_ind+(X_dim-24)], mb_ind))).reshape(size,1)
    hour = np.array(list(map(lambda mb_first_ind: hour_array[mb_first_ind+(X_dim-24)], mb_ind))).reshape(size,1)
    
    return mb_prices_day1, mb_prices_day2, weekday, hour


# generative network

X = tf.placeholder(tf.float32, shape=[None, discr_input_dim])
conditional = tf.placeholder(tf.float32, shape = [None, time_conditional_dim + 2])
z = tf.placeholder(tf.float32, shape=[None, gen_input_dim])

G_W1 = tf.Variable(xavier_init([gen_input_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
G_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[h_dim]))
G_W3 = tf.Variable(xavier_init([h_dim, time_predict_dim]))
G_b3 = tf.Variable(tf.zeros(shape=[time_predict_dim]))

theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]


# discriminative network

D_W1 = tf.Variable(xavier_init([discr_input_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


# functions defining operations in networks
def G(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)    # bekommt tensor übergeben (mbsize, z_inp_dim)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)    # bekommt tensor übergeben (mbsize, z_inp_dim)
    G_h3 = tf.nn.relu(tf.matmul(G_h2, G_W3) + G_b3)             # output layer hat die dimension (mbsize, X_dim)
    G_output = tf.sigmoid(G_h3)
    return G_h3, G_output


def D(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)    # bekommt tensor übergeben (mbsize, discr_inp_dim)
    out = tf.sigmoid(tf.matmul(D_h1, D_W2) + D_b2)  # output hat die dimension (mbsize, 1)
    return out

G_sample_wo_scaling, G_sample = G(z)       #  hat die dimension (mbsize,t_predict_dim)

D_real = D(X)         #  X hat die dimension (mbsize, t_predict_dim + t_conditional_dim + 2)

G_sample_cat = tf.concat([G_sample, conditional],1)

D_fake = D(G_sample_cat)  #  

D_target = 1./mb_size        #  this is the target of the discriminator net: exactly the number of real price series
G_target = 1./(mb_size*2)    #  this is the target of the gen net (for every time series 1/2)

Z = tf.reduce_sum(tf.exp(-D_real)) + tf.reduce_sum(tf.exp(-D_fake))

# the minimax game
D_loss = tf.reduce_sum(D_target * D_real) + log(Z)
G_loss = tf.reduce_sum(G_target * D_real) + tf.reduce_sum(G_target * D_fake) + log(Z)

reg_loss = tf.reduce_sum(tf.abs(G_W1)) + tf.reduce_sum(tf.abs(G_W2))

G_loss_total = tf.add(G_loss, reg_scale * reg_loss, name = "total_gen_loss")


# the training operation
D_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(D_loss, var_list=theta_D))
G_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(G_loss, var_list=theta_G))


# initialize the place holder nodes

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(no_iterations):
    
    X_mb_day1, X_mb_day2, weekday, hour = next_batch(mb_size)
    z_mb = sample_z(mb_size, z_dim)
    
    
    inputs_gen = np.concatenate((z_mb, X_mb_day1, weekday, hour), axis = 1)
    inputs_discr = np.concatenate((X_mb_day2, X_mb_day1, weekday, hour), axis = 1)
    conditional_array = np.concatenate((X_mb_day1, weekday, hour),axis = 1)
    
    _, D_loss_curr = sess.run(
        [D_solver, D_loss], feed_dict={X: inputs_discr, z: inputs_gen, conditional: conditional_array}
    )

    _, G_loss_curr, regularization_loss = sess.run(
        [G_solver, G_loss, reg_loss], feed_dict={X: inputs_discr, z: inputs_gen, conditional: conditional_array}
    )

    if it % 200 == 0:
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))
        print(regularization_loss)
        
        
        X_mb_day1, X_mb_day2, weekday, hour = next_batch(1)
        z_mb = sample_z(1, z_dim)
        
        inputs_gen = np.concatenate((z_mb, X_mb_day1, weekday, hour), axis = 1)
        inputs_discr = np.concatenate((X_mb_day2, X_mb_day1, weekday, hour), axis = 1)
        conditional_array = np.concatenate((X_mb_day1, weekday, hour),axis = 1)

        samples = sess.run(G_sample_wo_scaling, feed_dict={z: inputs_gen})
        
        samples_compl = np.concatenate((X_mb_day1, samples), axis = 1)
        
        real_data = np.concatenate((X_mb_day1, X_mb_day2), axis = 1)

       # print(np.ndarray.flatten(samples))
        fig = plt.plot(np.ndarray.flatten(samples_compl))
        fig = plt.plot(np.ndarray.flatten(real_data))
        plt.savefig('out/{}.png'
                    .format(str(i).zfill(4)), bbox_inches='tight')
        i += 1
        fig = plt.gcf()
        plt.close(fig)



Iter: 0; D_loss: 7.605; G_loss: 7.613
2401.8862
Iter: 200; D_loss: 7.224; G_loss: 7.721
2406.5186
Iter: 400; D_loss: 7.222; G_loss: 7.721
2407.2744
Iter: 600; D_loss: 7.222; G_loss: 7.721
2408.0547
Iter: 800; D_loss: 7.221; G_loss: 7.721
2408.3008
Iter: 1000; D_loss: 7.221; G_loss: 7.721
2408.4016
Iter: 1200; D_loss: 7.221; G_loss: 7.721
2408.5723
Iter: 1400; D_loss: 7.221; G_loss: 7.721
2408.6694
Iter: 1600; D_loss: 7.221; G_loss: 7.721
2408.7441
Iter: 1800; D_loss: 7.221; G_loss: 7.721
2408.7769
Iter: 2000; D_loss: 7.222; G_loss: 7.721
2408.838
Iter: 2200; D_loss: 7.221; G_loss: 7.721
2408.8718
Iter: 2400; D_loss: 7.221; G_loss: 7.721
2408.899
Iter: 2600; D_loss: 7.221; G_loss: 7.721
2408.9177
Iter: 2800; D_loss: 7.222; G_loss: 7.721
2408.938
Iter: 3000; D_loss: 7.222; G_loss: 7.721
2408.9534
Iter: 3200; D_loss: 7.221; G_loss: 7.721
2408.9673
Iter: 3400; D_loss: 7.221; G_loss: 7.721
2408.9968
Iter: 3600; D_loss: 7.221; G_loss: 7.721
2409.0105
Iter: 3800; D_loss: 7.221; G_loss: 7.721


KeyboardInterrupt: 

In [155]:
G_loss.shape

TensorShape([])

In [20]:
# testing !!!

X_dim = 48 

def next_batch(size):
    ind_vec = list(range(len(prices)-X_dim))
    ind_vec_shuffled = random.sample(ind_vec,len(ind_vec))

    mb_ind = ind_vec_shuffled[0:(size)]
    
    mb_prices_day1 = np.array(list(map(lambda mb_first_ind: prices[mb_first_ind:(mb_first_ind+(X_dim-24))], mb_ind)))
    
    mb_prices_day2 = np.array(list(map(lambda mb_first_ind: prices[(mb_first_ind+(X_dim-24)+1):(mb_first_ind+X_dim)], mb_ind)))
    
    return mb_prices_day1, mb_prices_day2

In [106]:
def next_batch(size):
    ind_vec = list(range(len(prices)-X_dim))
    ind_vec_shuffled = random.sample(ind_vec,len(ind_vec))

    mb_ind = ind_vec_shuffled[0:(size)]
    
    mb_prices_day1 = np.array(list(map(lambda mb_first_ind: prices[mb_first_ind:(mb_first_ind+(X_dim-24))], mb_ind)))   
    mb_prices_day2 = np.array(list(map(lambda mb_first_ind: prices[(mb_first_ind+(X_dim-24)):(mb_first_ind+X_dim)], mb_ind)))
    
    days_of_the_week_day1 = np.array(list(map(lambda mb_first_ind: day_of_week_array[mb_first_ind:(mb_first_ind+(X_dim-24))], mb_ind))) 
    days_of_the_week_day2 = np.array(list(map(lambda mb_first_ind: day_of_week_array[(mb_first_ind+(X_dim-24)):(mb_first_ind+X_dim)], mb_ind)))
    
    return mb_prices_day1, mb_prices_day2, days_of_the_week_day1, days_of_the_week_day2


a, b, c, d = next_batch(1)
print('Day 1: \n\n', a,'\n\n')
print('Day 2: \n\n',b,'\n\n')
print(a.shape)
print(b.shape)
print(c)
print(d)

Day 1: 

 [[23.08 23.06 23.06 25.61 28.65 31.99 34.05 38.7  38.98 34.08 30.65 26.76
  24.05 22.87 22.51 21.73 21.1  22.09 22.94 22.93 24.07 25.11 24.96 25.48]] 


Day 2: 

 [[25.16 24.54 23.48 23.42 24.01 28.73 36.36 39.85 42.57 38.93 35.94 31.27
  24.06 23.45 22.88 22.15 22.75 24.12 36.69 43.   42.53 39.43 33.5  30.71]] 


(1, 24)
(1, 24)
[[5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6]]
[[6 6 6 6 6 6 6 6 6 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [45]:
def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

c = sample_z(2,40)
c.shape

(2, 40)

In [49]:
A =np.concatenate((a,c),axis = 1)
A.shape

(2, 64)